You will need to install the following packages:
- `ipyleaflet`
- `requests`
- `xarray`
- `netcdf4`

In [ ]:
# Set up for JupyterLite
try:
    import piplite
    await piplite.install('ipyleaflet')
except ImportError:
    pass

In [ ]:
import ipyleaflet
from ipyleaflet.velocity import Velocity
import xarray as xr
from branca.colormap import linear

In [ ]:
center = [44.33956524809713, -130.60546875000003]
zoom = 3
m = ipyleaflet.Map(
    center=center,
    zoom=zoom,
    interpolation="nearest",
    basemap=ipyleaflet.basemaps.CartoDB.DarkMatter,
)

In [ ]:
import os

if not os.path.exists("wind-global.nc"):
    url = "https://github.com/benbovy/xvelmap/raw/master/notebooks/wind-global.nc"
    import requests

    r = requests.get(url)
    wind_data = r.content
    with open("wind-global.nc", "wb") as f:
        f.write(wind_data)

In [ ]:
ds = xr.open_dataset("wind-global.nc")

In [ ]:
display_options = {
    "velocityType": "Global Wind",
    "displayPosition": "bottomleft",
    "displayEmptyString": "No wind data",
}
wind = Velocity(
    data=ds,
    zonal_speed="u_wind",
    meridional_speed="v_wind",
    latitude_dimension="lat",
    longitude_dimension="lon",
    velocity_scale=0.01,
    max_velocity=20,
    colormap = linear.viridis,
    display_options=display_options
)

In [ ]:
colormap_control = ipyleaflet.ColormapControl(
    colormap=wind.colormap,
    value_min=wind.min_velocity,
    value_max=wind.max_velocity,
    position='topright',
    transparent_bg=False,
)
m.add(wind)
m